Les premiers éléments: 
- on crée le workspace 
- on charge les photos

Pour la suite de notre expérimentations, nous déciderons de ne garder qu'une légère partie des photos pour évaluer nos méthodes. 

In [ ]:
from azureml.core import Workspace, Datastore
from azureml.data.datapath import DataPath

from azureml.core import Workspace, Dataset, Datastore

subscription_id = 'e91c28d8'
resource_group = '-rg'
workspace_name = 'ml-experimentations'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "images_dmr")
dataset = Dataset.File.from_files(path=(datastore, 's66-pr-i1'))
mounted_path = dataset.mount()
mounted_path.start()

print(f"Datastore monté à : {mounted_path}")


Datastore monté à : <builtins.RslexURIMountContext object at 0x7fbe25815f70>


On récupère les photos. Nous avons environ 1,3M de photos. Nous réalisons quelques opérations pour nettoyer le tableau. 

In [ ]:
import pandas as pd 
df = pd.read_csv('df_final.csv', index_col=0)
df.niveau_1 = df.niveau_1.str.replace('/',' ')
df.niveau_2 = df.niveau_2.str.replace('/',' ')
df.niveau_3 = df.niveau_3.str.replace('/',' ')
df.chemin_photo = df.chemin_photo.str.strip()
df = df[df['chemin_photo'].str.endswith(('.jpeg', '.jpg'))]
df=df.replace('Autos motos vélos trottinettes...', 'Autos motos vélos...')
df.head()

Nous allons sélectionner un petit échantillon pour avancer sur nos expérimentations. 

In [11]:
def select_max_per_category(data, categorie, max_per_category=100):
    selected_data = []
    for category, group in data.groupby(categorie):
        # Mélanger et sélectionner jusqu'à max_per_category lignes
        selected_subset = group.sample(n=min(len(group), max_per_category), random_state=42)
        selected_data.append(selected_subset)
    return pd.concat(selected_data, ignore_index=True)

df_ = select_max_per_category(df, 'niveau_1')
df_.to_csv('echantillon_benchmark.csv')


### Préparation du modèle

In [6]:
df_=pd.read_csv('echantillon_benchmark.csv', index_col=0)
df_ = df_[['chemin_photo', 'niveau_1', 'niveau_2', 'niveau_3','categorie_fine']]

#On crée des catégories numériques pour que ce soit mieux compris par certains modèles
df_['n1_id'] = df_['niveau_1'].factorize()[0]
df_.head()

,chemin_photo,niveau_1,niveau_2,niveau_3,categorie_fine,n1_id
0,eac042288d63aad4852659dab3ebf692_4886226.jpeg,Activités commerciales et professionnelles,Étalages et terrasses à usage commercial,Gêne à la circulation Étalage excessif,Gêne à la circulation Étalage excessif,0
1,37a6bd7807a820088c8809a0d8d8f9e8_3122948.jpeg,Activités commerciales et professionnelles,Étalages et terrasses à usage commercial,Gêne à la circulation Étalage excessif,Gêne à la circulation Étalage excessif,0
2,3d9ca50bb90232cab637d0ae6d157fb0_4900775.jpeg,Activités commerciales et professionnelles,Étalages et terrasses à usage commercial,Nuisance sonore,Nuisance sonore,0
3,15a1263b8360d403f7aa4a0ea56cdb53_2424043.jpeg,Activités commerciales et professionnelles,Étalages et terrasses à usage commercial,Autre ...,Autre,0
4,1eb4392b8c9fa962c942a24744025e52_2330447.jpeg,Activités commerciales et professionnelles,Étalages et terrasses à usage commercial,Nuisance sonore ...,Nuisance sonore,0


On va séparer maintenant le jeu en deux sets: entraînement et test. 

In [8]:
X = df_[['chemin_photo', 'niveau_1', 'niveau_2', 'niveau_3','categorie_fine']]
y = df_['n1_id']
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

On va aller chercher les images maintenant. 

In [11]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Chemin d'accès aux images montées
source_dir = os.path.join(mounted_path.mount_point)
train_dir = "images_benchmark/train"

# Créer les dossiers pour le training et le test
os.makedirs(train_dir, exist_ok=True)


# Fonction pour organiser les fichiers
def organize_images(data_subset, destination_dir):
    for _, row in data_subset.iterrows():
        try:
            image_id = row["chemin_photo"]
            category = row["niveau_1"]
            category_dir = os.path.join(destination_dir, str(category))
            os.makedirs(category_dir, exist_ok=True)
            source_path = os.path.join(source_dir, image_id)
            dest_path = os.path.join(category_dir, image_id)
            # Copier l'image dans le dossier de catégorie
            if os.path.exists(source_path):
                shutil.copy(source_path, dest_path)
                print(f"Image ajoutée : {source_path}")
            else:
                print(f"Image manquante : {source_path}")
        except IsADirectoryError as e:
            print("Erreur directory",source_path )

# Organiser les images pour le training set et le test set
organize_images(Xtrain, train_dir)

print("Organisation des images terminée.")

Image ajoutée : /tmp/tmp19ippqu8/df1c0bc79a52b306ec325fb0a73e5919_5060463.jpeg
Image ajoutée : /tmp/tmp19ippqu8/43a171141fed501838e38e5c7ba7ee61_4949141.jpg
Image ajoutée : /tmp/tmp19ippqu8/0bd57b170316eb3fc211d17afb07557d_4333203.jpeg
Image ajoutée : /tmp/tmp19ippqu8/28a954222d2fbaa4cbf7480790c926be_2673520.jpeg
Image ajoutée : /tmp/tmp19ippqu8/1198183b3eb308cdd791fcfe752e22fb_2828205.jpeg
Image ajoutée : /tmp/tmp19ippqu8/0356afbf16840558bf02d5a1be33e867_2306267.jpeg
Image ajoutée : /tmp/tmp19ippqu8/5c2473e34866407f324a781ceaf19761_4987100.jpeg
Image ajoutée : /tmp/tmp19ippqu8/25699a70f843308347e370ef1459538d_3601955.jpeg
Image ajoutée : /tmp/tmp19ippqu8/12afcc0dc7bf5fb69780baa3b51d6dd4_2283391.jpeg
Image ajoutée : /tmp/tmp19ippqu8/510e37edf2f35a87541e4e29007958fd_4972200.jpeg
Image ajoutée : /tmp/tmp19ippqu8/e5d2ef7c80f2dd5828666bf38ccc7673_5149019.jpeg
Image ajoutée : /tmp/tmp19ippqu8/6b3fa0a7372d88bfe7d4fdfd5336bf39_5088682.jpeg
Image ajoutée : /tmp/tmp19ippqu8/0c5cf10399daf4eef87b

In [12]:
#Pareil pour le jeu de test
test_dir = "images_benchmark/test"
# Créer les dossiers pour le training et le test
os.makedirs(test_dir, exist_ok=True)
# Organiser les images pour le training set et le test set
organize_images(Xtest, test_dir)

print("Organisation des images terminée.")

Image ajoutée : /tmp/tmp19ippqu8/3067f1fbffadbf6395d45c6c3224b4b3_2726108.jpeg
Image ajoutée : /tmp/tmp19ippqu8/068e6f1a2885d82d3c02b32fe747132d_3968478.jpeg
Image ajoutée : /tmp/tmp19ippqu8/19124e5edb2cb4efbe50ffd8198bbfeb_2817252.jpeg
Image ajoutée : /tmp/tmp19ippqu8/570e0e2a808dc4e83a0f4fcd53de38d3_5060352.jpeg
Image ajoutée : /tmp/tmp19ippqu8/784576501c4d07874da8b7d8c9d2a4b2_4843089.jpeg
Image ajoutée : /tmp/tmp19ippqu8/39296efbe250b36157870a0502a0c782_2671626.jpeg
Image ajoutée : /tmp/tmp19ippqu8/29a5ad8931c2d56b4d665b116cf2c8c9_4288035.jpeg
Image ajoutée : /tmp/tmp19ippqu8/8c587a3963813bfd17adeaad09d39c36_5364493.jpeg
Image ajoutée : /tmp/tmp19ippqu8/1edc41f3e3143762098f5a16217733c6_3575382.jpeg
Image ajoutée : /tmp/tmp19ippqu8/2fc59506604b905142bfa359e0648b02_5093788.jpeg
Image ajoutée : /tmp/tmp19ippqu8/2ed867b9a7785c18cf545769c4728e0b_4603191.jpeg
Image ajoutée : /tmp/tmp19ippqu8/64a7431194c0312caf5006c1365d9530_5127340.jpeg
Image ajoutée : /tmp/tmp19ippqu8/5d6d44becdfbcc865ec

### Description des images

Nous allons encoder nos images en base64 avant. 

In [24]:
import io 
from PIL import Image
from tqdm import tqdm 
import base64
Xtrain['base64']=''
Xtest['base64']=''

# Chemin du répertoire parent et du dataset
dataset_dir = "images_benchmark/train"
dataset_dir2 = "images_benchmark/test"


for i in range(len(Xtrain)): 
    with open(os.path.join(dataset_dir, Xtrain['niveau_1'].iloc[i], Xtrain['chemin_photo'].iloc[i]), "rb") as f:
        data = f.read()
        data_base64 = base64.b64encode(data).decode("utf-8")  # Convertir en base64
    Xtrain['base64'].iloc[i]=data_base64

for i in range(len(Xtest)): 
    with open(os.path.join(dataset_dir2, Xtest['niveau_1'].iloc[i], Xtest['chemin_photo'].iloc[i]), "rb") as f:
        data = f.read()
        data_base64 = base64.b64encode(data).decode("utf-8")  # Convertir en base64
    Xtest['base64'].iloc[i]=data_base64

On récupère notre LLM et on joue sur le prompt. 

In [ ]:
import os  
import base64
from openai import AzureOpenAI  

endpoint = os.getenv("ENDPOINT_URL", "https://ope")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-classifier-images")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "c7") 

# Initialiser le client Azure OpenAI Service avec une authentification basée sur une clé    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)

Xtrain['desc-gpt4o']=''
Xtest['desc-gpt4o']=''
    
#On va générer les descriptions de toutes les photos du jeu d'entraînement. 
for i in tqdm(range(len(Xtrain))):
    #Préparer l’invite de conversation 
    chat_prompt = [
        {
            "role": "system",
            "content": "Vous êtes un assistant pour la Ville de Paris, spécialisée dans l'analyse de photos de situations parisiennes, soumises par des résidents parisiens. L'objectif est de réussir à fournir une description détaillée des photos envoyées aux agents de la Ville de Paris pour leur permettre de facilement catégoriser ces signalements & images. Vous fournirez une description détaillée et précise des images, tout en gardant bien ce contexte de Ville de Paris-urbanisme-signalements citoyens en tête. Cette description sera en anglais, et simplifiée ua maximum: ne décrit que ce que tu vois. "
        },    {
            "role": "user",
            "content": [
                        {
                            "type": "text",
                            "text": "here is an image", 
                            'image': Xtrain['base64'].iloc[i]
                        }
                    ] 
        },
    ] 
        
    # Inclure le résultat de la voix si la voix est activée  
    messages = chat_prompt 
        
    # Générer l’achèvement  
    completion = client.chat.completions.create(  
        model=deployment,
        messages=messages,
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
    )

    result= completion.choices[0].message.content
    result = " ".join([item.strip().lstrip('-').strip() for item in result.split('\n') if item.strip()])
    Xtrain['desc-gpt4o'].iloc[i]=result
    
Xtrain.to_csv('Xtrain.csv')

 56%|█████▌    | 447/800 [42:27<29:33,  5.02s/it]

In [ ]:
#On va générer les descriptions de toutes les photos du jeu d'entraînement. 
for i in tqdm(range(len(Xtest))):
    #Préparer l’invite de conversation 
    chat_prompt = [
        {
            "role": "system",
            "content": "Vous êtes un assistant pour la Ville de Paris, spécialisée dans l'analyse de photos de situations parisiennes, soumises par des résidents parisiens. L'objectif est de réussir à fournir une description détaillée des photos envoyées aux agents de la Ville de Paris pour leur permettre de facilement catégoriser ces signalements & images. Vous fournirez une description détaillée et précise des images, tout en gardant bien ce contexte de Ville de Paris-urbanisme-signalements citoyens en tête. Cette description sera en anglais, et simplifiée ua maximum: ne décrit que ce que tu vois. "
        },    {
            "role": "user",
            "content": [
                        {
                            "type": "text",
                            "text": "here is an image", 
                            'image': Xtrain['base64'].iloc[i]
                        }
                    ] 
        },
    ] 
        
    # Inclure le résultat de la voix si la voix est activée  
    messages = chat_prompt 
        
    # Générer l’achèvement  
    completion = client.chat.completions.create(  
        model=deployment,
        messages=messages,
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
    )

    result= completion.choices[0].message.content
    result = " ".join([item.strip().lstrip('-').strip() for item in result.split('\n') if item.strip()])
    Xtest['desc-gpt4o'].iloc[i]=result
Xtest.to_csv('Xtest.csv')

### Traitement de texte

Nous avons donc un jeu de données d'entraînement et de test, avec des descriptions détaillées. 

In [29]:
import pandas as pd 
#On va récupérer les datasets
Xtrain= pd.read_csv('Xtrain.csv')
Xtest = pd.read_csv('Xtest.csv')
df_=pd.read_csv('echantillon_benchmark.csv', index_col=0)
df_ = df_[['chemin_photo', 'niveau_1', 'niveau_2', 'niveau_3','categorie_fine']]

#On crée des catégories numériques pour que ce soit mieux compris par certains modèles
df_['n1_id'] = df_['niveau_1'].factorize()[0]


In [30]:
#On récupère les labels, car ils n'étaient pas sauvegardés
ytrain = pd.merge(Xtrain, df_, on='chemin_photo', how='inner')['n1_id']
ytest=pd.merge(Xtest, df_, on='chemin_photo', how='inner')['n1_id']

Préparons nos descriptions. 

In [37]:
#Preprocessing 
import string 

def clean_text(text):
    text = text.lower()  # Conversion en minuscules
    text = ''.join([char for char in text if char not in string.punctuation])  # Suppression de la ponctuation
    return text
Xtrain['desc-gpt4o']=Xtrain['desc-gpt4o'].apply(clean_text)
Xtest['desc-gpt4o']=Xtest['desc-gpt4o'].apply(clean_text)

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(stop_words='english')
# We transform each complaint into a vector
Xtrainvect = tfidf.fit_transform(Xtrain['desc-gpt4o']).toarray()
Xtestvect = tfidf.transform(Xtest['desc-gpt4o']).toarray()

### Création des modèles

In [41]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

model = LogisticRegression()
model.fit(Xtrainvect, ytrain)
ypred=model.predict(Xtestvect)
# Classification report
print('ttttCLASSIFICATIION METRICSn')
print(classification_report(ytest, ypred, 
                                    target_names= Xtrain['niveau_1'].unique()))

ttttCLASSIFICATIION METRICSn
                                            precision    recall  f1-score   support

Activités commerciales et professionnelles       0.05      0.05      0.05        20
                         Mobiliers urbains       0.08      0.15      0.11        13
                         Objets abandonnés       0.11      0.07      0.09        27
                   Voirie et espace public       0.17      0.19      0.18        21
                Arbres végétaux et animaux       0.07      0.13      0.09        15
                   Éclairage   Électricité       0.28      0.23      0.25        22
   Graffitis tags affiches et autocollants       0.00      0.00      0.00        25
                                       Eau       0.04      0.08      0.05        13
                      Autos motos vélos...       0.00      0.00      0.00        23
                                  Propreté       0.10      0.10      0.10        21

                                  accuracy   